In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist
import re
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import *
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamroth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
path = os.getcwd()
path

'/Users/adamroth/Documents/flat_fe_work/mod4/week13/NLP---Twitter-Sentiment/notebooks/AR_notebooks'

### basic info

In [8]:
raw_data = pd.read_csv('../../data/judge-1377884607_tweet_product_company.csv', encoding = 'latin1')

In [9]:
raw_data.sample(15)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
7937,UberGenius: Uber Is Taking Control of SXSW Ped...,NaN,No emotion toward brand or product
2239,Marissa Mayer's preso at #sxsw was good. Big a...,NaN,No emotion toward brand or product
5611,RT @mention Check out the FREE @mention Sample...,Other Apple product or service,Positive emotion
7183,Apple &quot;Pops Up&quot; Store for SXSW Event...,NaN,No emotion toward brand or product
7233,checking out iPad 2 @mention the #sxsw pop-up ...,iPad,Positive emotion
5735,RT @mention For those #notatSXSW (or at #SXSW)...,NaN,No emotion toward brand or product
8817,#fsw #sxsw for those of you who wanted to know...,NaN,No emotion toward brand or product
367,ÛÏ@mention Best thing I've heard this weekend...,iPad,Negative emotion
2592,The Google / ACLU party tonight at #sxsw is th...,Google,Positive emotion
4225,kinda jealous of the ubiquity of ipad's here.....,iPad,Positive emotion


In [10]:
raw_data['emotion_in_tweet_is_directed_at'].sample(10)

2778                               iPad
1541                                NaN
2687    Other Google product or service
4704                                NaN
5683                                NaN
3037                              Apple
1540                               iPad
1253                             Google
4396                                NaN
146                                iPad
Name: emotion_in_tweet_is_directed_at, dtype: object

In [11]:
raw_data['emotion_in_tweet_is_directed_at'].unique()

array(['iPhone', 'iPad or iPhone App', 'iPad', 'Google', nan, 'Android',
       'Apple', 'Android App', 'Other Google product or service',
       'Other Apple product or service'], dtype=object)

In [12]:
raw_data.isna()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
9088,False,False,False
9089,False,True,False
9090,False,True,False
9091,False,True,False


In [13]:
raw_data.dtypes

tweet_text                                            object
emotion_in_tweet_is_directed_at                       object
is_there_an_emotion_directed_at_a_brand_or_product    object
dtype: object

In [14]:
raw_data['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [15]:
#lots of nan's in this column, shouldn't matter while training model though

In [16]:
raw_data['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [17]:
#not many negatives, might need to mess with that, could use something like smote, might remove all from 'I cant tell'

### Tokenization

In [18]:
raw_data.iloc[i].tweet_text

NameError: name 'i' is not defined

In [19]:
raw_data.iloc[0].tweet_text.split()

['.@wesley83',
 'I',
 'have',
 'a',
 '3G',
 'iPhone.',
 'After',
 '3',
 'hrs',
 'tweeting',
 'at',
 '#RISE_Austin,',
 'it',
 'was',
 'dead!',
 'I',
 'need',
 'to',
 'upgrade.',
 'Plugin',
 'stations',
 'at',
 '#SXSW.']

In [20]:
test_list = raw_data.iloc[0].tweet_text.split()
type(test_list)

list

In [21]:
df = raw_data
df['split_text'] = 0
df.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product', 'split_text'],
      dtype='object')

In [22]:
raw_data['tweet_text'][5]

'@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conference http://ht.ly/49n4M #iear #edchat #asd'

In [23]:
for i in range(0,len(raw_data)):
    text = raw_data['tweet_text'][i]
    split_text = str(text).split()
    df['split_text'][i] = split_text
    
    



/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [24]:
df['split_text'][4]

['@sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 '#SXSW:',
 'Marissa',
 'Mayer',
 '(Google),',
 'Tim',
 "O'Reilly",
 '(tech',
 'books/conferences)',
 '&amp;',
 'Matt',
 'Mullenweg',
 '(Wordpress)']

In [25]:
import string
punctuation = string.punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [26]:
def clean_punct(string):

    clean_text_list = [''.join(ch for ch in s if ch not in punctuation) for s in string]
    
    #clean_text = ""
    
    #return (clean_text.join(clean_text_list))
    return clean_text_list
#cleans up punctuation

In [27]:
string = "@sxtxstate"
clean_punct(string)
print(clean_punct(string))

['', 's', 'x', 't', 'x', 's', 't', 'a', 't', 'e']


In [28]:
clean_punct(df['split_text'][4])

['sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 'SXSW',
 'Marissa',
 'Mayer',
 'Google',
 'Tim',
 'OReilly',
 'tech',
 'booksconferences',
 'amp',
 'Matt',
 'Mullenweg',
 'Wordpress']

In [29]:
# def clean_column(df['column']):
#     for i in range(0, len(df)):
#         thing = df['column'][i]
#         clean_string = str(thing)
#         return 

In [30]:
df['no_punct'] = 0

In [31]:
for i in range(0, len(df)):
    has_punct = df['split_text'][i]
    no_punct = clean_punct(has_punct)
    df['no_punct'][i] = no_punct

/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [32]:
df['no_punct'][1]

['jessedee',
 'Know',
 'about',
 'fludapp',
 '',
 'Awesome',
 'iPadiPhone',
 'app',
 'that',
 'youll',
 'likely',
 'appreciate',
 'for',
 'its',
 'design',
 'Also',
 'theyre',
 'giving',
 'free',
 'Ts',
 'at',
 'SXSW']

In [33]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,split_text,no_punct
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,"[.@wesley83, I, have, a, 3G, iPhone., After, 3...","[wesley83, I, have, a, 3G, iPhone, After, 3, h..."
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,"[@jessedee, Know, about, @fludapp, ?, Awesome,...","[jessedee, Know, about, fludapp, , Awesome, iP..."
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,"[@swonderlin, Can, not, wait, for, #iPad, 2, a...","[swonderlin, Can, not, wait, for, iPad, 2, als..."
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,"[@sxsw, I, hope, this, year's, festival, isn't...","[sxsw, I, hope, this, years, festival, isnt, a..."
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,"[@sxtxstate, great, stuff, on, Fri, #SXSW:, Ma...","[sxtxstate, great, stuff, on, Fri, SXSW, Maris..."
